In [ ]:
import sys
import matplotlib
matplotlib.rcParams["pdf.fonttype"] = 42
matplotlib.rcParams["ps.fonttype"] = 42
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
# from datetime import datetime

import numpy as np
# import numpy.random as random
import pandas as pd
import scanpy as sc
import os
import anndata as ad
import sklearn
# import louvain
# import torch

# from scvi.dataset import Dataset10X, CsvDataset, AnnDatasetFromAnnData, CellMeasurement, LoomDataset, DownloadableAnnDataset
# from scvi.dataset.dataset import GeneExpressionDataset
# from scvi.inference import TotalPosterior, TotalTrainer, load_posterior
# from scvi.models import SCANVI, TOTALVI
# from scvi import set_seed

# from umap import UMAP

# # Control UMAP numba warnings
# import warnings; warnings.simplefilter('ignore')

# %matplotlib inline

# set_seed(123)

# use_cuda = True
# show_plot = True
# test_mode = False

In [ ]:
def normalizeEmbedding(arr):   
    # scaling to [-1, 1]
    min = np.min(arr)
    diff = np.max(arr) - min
    arr[:, 0] = (2 * ((arr[:, 0] - min) / diff)) - 1
    arr[:, 1] = (2 * ((arr[:, 1] - min) / diff)) - 1
    return arr

def scaling(arr):
    # then scaling to [-1, 1]
    min = np.min(arr)
    diff = np.max(arr) - min
    arr[:, 0] = (2 * ((arr[:, 0] - min) / diff)) - 1
    arr[:, 1] = (2 * ((arr[:, 1] - min) / diff)) - 1
    return arr

def scaling_zero(arr):
    min = np.min(arr)
    diff = np.max(arr) - min
    arr[:, 0] = ((arr[:, 0] - min) / diff)
    arr[:, 1] = ((arr[:, 1] - min) / diff)
    return arr

### Load umap CD45-

In [ ]:
### Load adata
import pickle
baseFolder='./'
adataCD45neg = pickle.load(open(baseFolder+"post_adata_mouseCD45neg.pkl", "rb"))

In [ ]:
np.var(adataCD45neg.layers["norm_genes"], axis=0)

In [ ]:
### Latent space extracted from TotalVI model
# adataCD45neg.obsm["X_totalVI"]

### This latent space was used to calculate the UMAP:
## sc.pp.neighbors(adataCD45neg, use_rep="X_totalVI", n_neighbors=30, metric="correlation")
## sc.tl.umap(adataCD45neg, min_dist=0.2)

In [ ]:
# these are the interesting metadata features
# sampleID is the same as sampleName, sample are integers
adataCD45neg.obs['sampleName'] = adataCD45neg.obs['sampleName'].astype('category')
adataCD45neg.obs['louvain'] = adataCD45neg.obs['louvain'].astype('category')
adataCD45neg.obs['type'] = adataCD45neg.obs['type'].astype('category')
# annotID is the same as annot
adataCD45neg.obs['annot'] = adataCD45neg.obs['annot'].astype('category')

In [ ]:
sc.pp.pca(adataCD45neg, n_comps=50, zero_center=True)

In [ ]:
adataCD45neg.obsm["PCA_0"] = normalizeEmbedding(adataCD45neg.obsm["X_pca"][:, 0:2])
adataCD45neg.obsm["UMAP_0"] = normalizeEmbedding(adataCD45neg.obsm["X_umap"])
adataCD45neg.uns["methods"] = {
    "UMAP": 1,
    "PCA": 1,
}
adataCD45neg.uns["UMAP_0"] = {}
adataCD45neg.uns["PCA_0"] = {}

In [ ]:
delete_obs = [ob for ob in adataCD45neg.obs_keys() if "adt" in ob]
for delob in delete_obs:
    del adataCD45neg.obs[delob]

In [ ]:
# cleanup adata
del adataCD45neg.obsm['X_umap']
del adataCD45neg.uns['pca']

In [ ]:
# center the data to compute 'correlation' distance in all DR
adataCD45neg.obsm['X_totalVI'] = adataCD45neg.obsm['X_totalVI'] - np.mean(adataCD45neg.obsm['X_totalVI'], axis=1, keepdims=True)

In [ ]:
adataCD45neg.uns['methods']['tSNE_skrodzki'] = 5

for i in range(5):
    with open(os.path.join(baseFolder, 'embeddings', f'tsne_skrodzki_{i}.csv')) as f:
        adataCD45neg.uns[f'tSNE_skrodzki_{i}'] = {}
        embedding = np.loadtxt(f, delimiter=",")
        embedding_arr = np.asarray([embedding[:, 0], embedding[:, 1]], dtype=np.float32).T
        adataCD45neg.obsm[f"tSNE_skrodzki_{i}"] = normalizeEmbedding(embedding_arr)

In [ ]:
adataCD45neg

In [ ]:
adataCD45neg.write_h5ad('mouseCD45neg_embeddings.h5ad', compression='gzip')

In [ ]:
### Plot umap
tmp=sc.pl.umap(
    adataCD45neg, 
    color=["louvain","annot"],
    alpha=0.9,
    legend_loc="on data",
#     legend_loc="right margin",
    return_fig=True
)

In [ ]:
### Plot umap
tmp=sc.pl.umap(
    adataCD45neg, 
    color=["type","sampleName"],
    alpha=0.9,
    legend_loc="right margin",
    return_fig=True
)

In [ ]:
sc.pl.embedding(adataCD45neg, basis='UMAP_0', color='annot')

In [ ]:
filepath = 'mouseCD45neg_embeddings_neighbors_quality.h5ad'
baseFolder='./'
adataCD45neg = ad.read_h5ad(os.path.join(baseFolder, filepath))

In [ ]:
adataCD45neg.obsm["PCA_norm_genes_0"] = normalizeEmbedding(sc.tl.pca(adataCD45neg.layers["norm_genes"]))
adataCD45neg.uns["methods"]['PCA_norm_genes'] = 1
adataCD45neg.uns["PCA_norm_genes_0"] = {}

In [ ]:
adataCD45neg.write_h5ad('mouseCD45neg_embeddings_neighbors_quality.h5ad', compression='gzip')

### Load umap fibroblasts

In [ ]:
### Load adata
import pickle
baseFolder='./'
adataFibro = pickle.load(open(baseFolder+"post_adata_mouseFibroblasts.pkl", "rb"))
adataFibro.n_obs # 5430 cells but when filtering the whole dataset, we get 5431 cells?

In [ ]:
# which cell is missing?

fibro_subset = np.nonzero((adataCD45neg.obs['annot'] == 'Stellate cells') | 
               (adataCD45neg.obs['annot'] == 'Capsular Fibroblasts') |
               (adataCD45neg.obs['annot'] == 'Mesothelial cells') |
               (adataCD45neg.obs['annot'] == 'Fibroblasts'))[0]

for cell in fibro_subset:
    if len(np.nonzero(adataCD45neg.obs['cell'][cell] == adataFibro.obs['cell'])[0]) == 0:
        print(f"{cell} is not in adataFibro")
        
adataCD45neg.obs.iloc[55424]
# 55424 is not in adataFibro (the only cell of sample CS141)

In [ ]:
# indices of adataFibro in the adataCD45neg data
adataCD45neg_fibro_indices = []

for cell in adataFibro.obs['cell']:
    adataCD45neg_fibro_indices.append(np.nonzero(adataCD45neg.obs['cell'] == cell)[0][0])

In [ ]:
adataFibro.obsm["UMAP_global_0"] = normalizeEmbedding(adataCD45neg.obsm['UMAP_0'][adataCD45neg_fibro_indices, :])

In [ ]:
# center the data to compute 'correlation' distance in all DR
np.unique(adataFibro.obsm['X_totalVI'].mean(axis=1))
adataFibro.obsm['X_totalVI'] = adataFibro.obsm['X_totalVI'] - np.mean(adataFibro.obsm['X_totalVI'], axis=1, keepdims=True)

In [ ]:
sc.pp.pca(adataFibro, n_comps=2, zero_center=True)
adataFibro.obsm["PCA_norm_genes_0"] = sc.tl.pca(adataFibro.layers["norm_genes"])

In [ ]:
adataFibro.obsm["PCA_0"] = normalizeEmbedding(adataFibro.obsm["X_pca"][:, 0:2])
adataFibro.obsm["UMAP_0"] = normalizeEmbedding(adataFibro.obsm["X_umap"])
del adataFibro.obsm["X_pca"]

In [ ]:
# also store coordinates from UMAP of whole dataset
adataFibro.uns["methods"] = {
    "UMAP": 1,
    "UMAP_global": 1,
    "PCA": 1,
    "PCA_norm_genes": 1,
}
adataFibro.uns["UMAP_0"] = {}
adataFibro.uns["UMAP_global_0"] = {}
adataFibro.uns["PCA_0"] = {}
adataFibro.uns["PCA_norm_genes_0"] = {}

In [ ]:
### Latent space extracted from scVI model
# adataFibro.obsm["X_totalVI"]

### This latent space was used to calculate the UMAP:
## sc.pp.neighbors(adataFibro, use_rep="X_totalVI", n_neighbors=30, metric="correlation")
## sc.tl.umap(adataFibro, min_dist=0.2)

In [ ]:
### Change the annotation a bit
adataFibro.obs=adataFibro.obs.replace('Fibroblast 1','CV Fibroblasts')
adataFibro.obs=adataFibro.obs.replace('Fibroblast 2','Bile-duct Fibroblasts')

In [ ]:
adataFibro.obs['sampleName'] = adataFibro.obs['sampleName'].astype('category')
adataFibro.obs['louvain'] = adataFibro.obs['louvain'].astype('category')
adataFibro.obs['type'] = adataFibro.obs['type'].astype('category')
adataFibro.obs['annot'] = adataFibro.obs['annot'].astype('category')

In [ ]:
adataFibro.obs['annot'].unique()

In [ ]:
### Plot umap
tmp=sc.pl.umap(
    adataFibro, 
    color=["louvain","annot"],
    alpha=0.9,
    legend_loc="on data",
#     legend_loc="right margin",
    return_fig=True
)

In [ ]:
sc.pl.embedding(adataFibro, basis='UMAP_global_0', color='annot')

In [ ]:
### Plot umap
tmp=sc.pl.umap(
    adataFibro, 
    color=["type","sampleName"],
    alpha=0.9,
    legend_loc="right margin",
    return_fig=True
)

In [ ]:
adataFibro.write_h5ad('mouseFibro_embeddings.h5ad', compression='gzip')

In [ ]:
filepath = 'mouseFibro_embeddings.h5ad'
baseFolder='./'
adataFibro = ad.read_h5ad(os.path.join(baseFolder, filepath))

In [ ]:
adataFibro.uns["methods"] = {
    "UMAP": 1,
    "UMAP_global": 1,
    "PCA": 1,
}
adataFibro.uns['methods']['tSNE_skrodzki'] = 5

for i in range(5):
    with open(os.path.join(baseFolder, 'embeddings', f'fibro_tsne_skrodzki_{i}.csv')) as f:
        adataFibro.uns[f'tSNE_skrodzki_{i}'] = {}
        embedding = np.loadtxt(f, delimiter=",")
        embedding_arr = np.asarray([embedding[:, 0], embedding[:, 1]], dtype=np.float32).T
        adataFibro.obsm[f"tSNE_skrodzki_{i}"] = normalizeEmbedding(embedding_arr)

adataFibro.write_h5ad('mouseFibro_embeddings.h5ad', compression='gzip')

In [ ]:
filepath = 'mouseFibro_embeddings_neighbors_quality.h5ad'
baseFolder='./'
adataFibro = ad.read_h5ad(os.path.join(baseFolder, filepath))
adataFibro

In [ ]:
adataFibro.obsm["PCA_norm_genes_0"] = normalizeEmbedding(sc.tl.pca(adataFibro.layers["norm_genes"]))
adataFibro.uns["methods"]['PCA_norm_genes'] = 1
adataFibro.uns["PCA_norm_genes_0"] = {}

In [ ]:
adataFibro.uns["UMAP_0"]

In [ ]:
adataFibro.obsm["UMAP_0"] = normalizeEmbedding(adataFibro.obsm["UMAP_0"])
adataFibro.obsm["UMAP_global_0"] = normalizeEmbedding(adataFibro.obsm["UMAP_global_0"])
adataFibro.obsm["PCA_0"] = normalizeEmbedding(adataFibro.obsm["PCA_0"])

In [ ]:
adataFibro.write_h5ad('mouseFibro_embeddings_neighbors_quality.h5ad', compression='gzip')

In [ ]:
adataFibro.uns['UMAP_0'] = {}
adataFibro.uns['UMAP_global_0'] = {}
adataFibro.uns['PCA_0'] = {}

In [ ]:
from sklearn.metrics import pairwise_distances

def get_cluster_medians(cluster_labels, embedding_name):
    medians = []
    for cluster in list(adataFibro.obs['louvain'].value_counts(ascending=False, sort=True).keys()):
        ind = list(np.nonzero(cluster_labels == cluster)[0])
        if len(ind) == 0:
            continue
        medians.append(np.median(adataFibro.obsm[embedding_name][ind,], axis=0))
    medians = np.row_stack(medians)
    return medians

hd_medians = get_cluster_medians(adataFibro.obs['louvain'], 'X_totalVI')
hd_median_distances = pairwise_distances(hd_medians, metric='cosine')

In [ ]:
sklearn.metrics.pairwise.PAIRWISE_DISTANCE_FUNCTIONS

In [ ]:
umap_medians = get_cluster_medians(adataFibro.obs['louvain'], 'UMAP_0')
umap_median_distances = pairwise_distances(umap_medians, metric='euclidean')
umap_corr = []
for i in range(hd_median_distances.shape[0]):
    umap_corr.append(stats.spearmanr(hd_median_distances[i, :], b=umap_median_distances[i, :], axis=0).correlation)
umap_corr = np.row_stack(umap_corr)

In [ ]:
tsne_medians = get_cluster_medians(adataFibro.obs['louvain'], 'tSNE_skrodzki_0')
tsne_median_distances = pairwise_distances(tsne_medians, metric='euclidean')
tsne_corr = []
for i in range(hd_median_distances.shape[0]):
    tsne_corr.append(stats.spearmanr(hd_median_distances[i, :], b=tsne_median_distances[i, :], axis=0).correlation)
tsne_corr = np.row_stack(tsne_corr)

In [ ]:
print(dict(zip(range(23), hd_median_distances[22,])))

In [ ]:
adataFibro.layers['norm_genes']
np.argwhere(adataFibro.var.index == 'Tcea1')[0][0]